So far we've learnt how to scrape the web, and how to make a request for information from an API. Some websites make APIs even easier. Check out [RapidAPI](https://rapidapi.com/) they take care of writing most of the code for you.

We will use the [AeroDataBox API](https://rapidapi.com/aedbx-aedbx/api/aerodatabox/), which can retrieve all sorts of information about flights and airports. We will show you how to retrieve information about the airports, and then it's up to you to apply this, along with what you've already learnt this week, to **produce a function, which retrieves tomorrows flight information for the major airports in the cities you web scraped**.

In [6]:
import pandas as pd
import requests
from keys import AeroDatabox
from keys import sql
import sqlalchemy

On the left hand side of the AeroDataBox API page, you'll see a list of options for information that you can retrieve:
> - Flights API
- Subsciption / PUSH API
- Airport API
- Aircraft API
- Healthcheck & Status API

1. We want to select `Airport API`

2. Then within Airport API we want to select `Search airports by location`

3. Now in the middle third you'll want to enter the `latitude` and `longitude` of any city to test... we chose Berlin: latitude 52.31 longitude 13.24. Next we changed the `radiusKM` to only 50km. And finally set `withFlightInfoOnly` to true, so it will only return airports which have flight data (scheduled or live) available.

4. On the right hand third of the screen you should see a block of code that looks pretty unfamiliar. This is because by default the code is probably set to *(Node.js) Axios*. However, we have the power to change this to familiar python. Select the dropdown box at the top of the code and select `python > requests`.

Now you can copy the code to your notebook and it should look a little something like the cell below:

In [8]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

querystring = {"lat":"52.31","lon":"13.24","radiusKm":"50","limit":"10","withFlightInfoOnly":"true"}

headers = {
	"x-rapidapi-key": AeroDatabox,
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'searchBy': {'lat': 52.31, 'lon': 13.24}, 'count': 1, 'items': [{'icao': 'EDDB', 'iata': 'BER', 'name': 'Berlin Brandenburg', 'shortName': 'Brandenburg', 'municipalityName': 'Berlin', 'location': {'lat': 52.35139, 'lon': 13.493889}, 'countryCode': 'DE', 'timeZone': 'Europe/Berlin'}]}


Let's view the response as `.json()` so that it's easier to read

In [10]:
response.json()

{'searchBy': {'lat': 52.31, 'lon': 13.24},
 'count': 1,
 'items': [{'icao': 'EDDB',
   'iata': 'BER',
   'name': 'Berlin Brandenburg',
   'shortName': 'Brandenburg',
   'municipalityName': 'Berlin',
   'location': {'lat': 52.35139, 'lon': 13.493889},
   'countryCode': 'DE',
   'timeZone': 'Europe/Berlin'}]}

In [12]:
from IPython.display import JSON

In [14]:
JSON(response.json())

<IPython.core.display.JSON object>

In [16]:
response.json().keys()

dict_keys(['searchBy', 'count', 'items'])

In [18]:
response.json()['items'][0]['icao']

'EDDB'

In [20]:
for item in response.json()['items']:
    print(item)

{'icao': 'EDDB', 'iata': 'BER', 'name': 'Berlin Brandenburg', 'shortName': 'Brandenburg', 'municipalityName': 'Berlin', 'location': {'lat': 52.35139, 'lon': 13.493889}, 'countryCode': 'DE', 'timeZone': 'Europe/Berlin'}


Let's now use this for the latitude and longitude of multiple cities

In [32]:
def icao_airport_codes():

    schema = "gans_local" # The name of your database
    host = "127.0.0.1"
    user = "root"
    password = sql # Your MySQL password
    port = 3306

    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    cities_df = pd.read_sql("cities", con=connection_string)

    list_for_df = []

    for _, city in cities_df.iterrows():
        latitude = city["Latitude"]
        longitude = city["Longitude"]
        city_id = city["City_id"]

        url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

        querystring = {"lat":latitude,"lon":longitude,"radiusKm":"50","limit":"5","withFlightInfoOnly":"true"}
        headers = {
          "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
          "X-RapidAPI-Key": AeroDatabox #USE THE PROVIDED KEY
        }

        response = requests.get(url, headers=headers, params=querystring)

        for item in response.json()['items']:
            airports_data = {
            "city_id": city_id,
            "icao": item.get("icao", None),
            "municipality_name": item.get("municipalityName", None)
        }


        list_for_df.append(airports_data)

    airports_df = pd.DataFrame(list_for_df)

    return airports_df

In [34]:
airports_df = icao_airport_codes()
airports_df

,city_id,icao,municipality_name
0,1,EDDB,Berlin
1,2,EDDH,Hamburg
2,3,EDDM,Munich


In [ ]:
airports_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   city_id            3 non-null      int64 
 1   icao               3 non-null      object
 2   municipality_name  3 non-null      object
dtypes: int64(1), object(2)
memory usage: 200.0+ bytes


### Create "airports" table in the database

```sql
CREATE TABLE airports(
    city_id INT NOT NULL,
    icao VARCHAR(10),
    municipality_name VARCHAR(255),
    PRIMARY KEY (icao),
    FOREIGN KEY (city_id) REFERENCES cities(City_id)
);
```

### Push the "airports_to_db" to the empty "airports" table in the database

In [35]:
schema = "gans_local" # The name of your database
host = "127.0.0.1"
user = "root"
password = sql # Your MySQL password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [37]:
airports_df.to_sql('airports',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

In [24]:
pd.read_sql("airports", con=connection_string)

,city_id,icao,municipality_name


### **Challenge:** Arrivals information
Using what you have been shown above, plus the skills you've learnt in the last couple of days:
1. In `AeroDataBox API` use the `Flight API` > `FIDS/Schedules: Airport departures and arrivals by airport ICAO code` section
2. Fill out the parameters in the middle third and then copy the `python: requests` code from the right hand third
3. Explore the data you get back. What would be useful in your DataFrame and what can be excluded? Remember Gans wants to know about when people are arriving in the city
4. Make a DataFrame from the information you see as important
5. Condense everything you did above into a function that can take a list of ICAO codes as an input, and as an output gives you a DataFrame with the information for **tomorrows arrivals**

In [11]:
import requests

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EEDB/2024-10-09T00:00/2024-10-09T11:00"

querystring = {"withLeg":"true","direction":"Both","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
## push airport backbone to the database

In [26]:
from keys import gcp
schema = "gans_global" # The name of your database
host = "35.187.50.113"
user = "root"
password = gcp # Your MySQL password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [36]:
airports_df.to_sql('airports',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3